来源： [b站刘二大人--多分类问题](https://www.bilibili.com/video/BV1Y7411d7Ys?p=9)
# 0. Test CrossEntropyLoss

In [1]:
import torch
criterion = torch.nn.CrossEntropyLoss()
Y = torch.LongTensor([2, 0, 1])

Y_pred1 = torch.Tensor([
    [0.1, 0.2, 0.9],
    [1.1, 0.1, 0.2],
    [0.2, 2.1, 0.1]
])
Y_pred2 = torch.Tensor([
    [0.8, 0.2, 0.3],
    [0.2, 0.3, 0.5],
    [0.2, 0.2, 0.5]
])

l1 = criterion(Y_pred1, Y)
l2 = criterion(Y_pred2, Y)
print("Batch Loss1 = ", l1.data)
print("Batch Loss2 = ", l2.data)

Batch Loss1 =  tensor(0.4966)
Batch Loss2 =  tensor(1.2389)


# 1. Implementation for MINIST
## 1.1 Prepare Dataset

In [2]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
]) # 归一化,均值和方差
 
train_dataset = datasets.MNIST(root='./dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='./dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

## 1.2 Design Model

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        # 最后一层不需要激活，注意CrossEntropyLoss包含了logSoftmax+NLLloss
        return self.l5(x)

model = Net()


## 1.3 Construct Loss ans Optimizer

In [5]:
# 查看模型参数shape

print(train_loader)
print(test_loader)
print(model.parameters())
# 参数说明
# 第一层的参数：
layer1_weight = model.l1.weight.data
layer1_bias = model.l1.bias.data
# print("layer1_weight", layer1_weight)
print("layer1_weight.shape", layer1_weight.shape)
# print("layer1_bias", layer1_bias)
print("layer1_bias.shape", layer1_bias.shape)

print("layer2_weight.shape", model.l2.weight.data.shape)
print("layer2_bias.shape", model.l2.bias.data.shape)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

<generator object Module.parameters at 0x7f82a5f56d58>
layer1_weight.shape torch.Size([512, 784])
layer1_bias.shape torch.Size([512])
layer2_weight.shape torch.Size([256, 512])
layer2_bias.shape torch.Size([256])


## 1.4 Train and Test

In [6]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()

        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1) # dim = 1 列是第0个维度，行是第1个维度
            total += labels.size(0)
            correct += (predicted == labels).sum().item() # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))

for epoch in range(5):
    train(epoch)
    test()


[1,   300] loss: 2.260
[1,   600] loss: 1.088
[1,   900] loss: 0.429
accuracy on test set: 89 % 
[2,   300] loss: 0.319
[2,   600] loss: 0.264
[2,   900] loss: 0.228
accuracy on test set: 94 % 
[3,   300] loss: 0.187
[3,   600] loss: 0.165
[3,   900] loss: 0.157
accuracy on test set: 95 % 
[4,   300] loss: 0.130
[4,   600] loss: 0.126
[4,   900] loss: 0.115
accuracy on test set: 96 % 
[5,   300] loss: 0.096
[5,   600] loss: 0.095
[5,   900] loss: 0.099
accuracy on test set: 96 % 


## 1.5 理解维度

In [7]:

for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
    optimizer.zero_grad()
    print("target.shape", target.shape, target)
    # forward + backward + update
    outputs = model(inputs)
    print("output.shape", outputs.shape, outputs[0])
    break
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:
        print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
        running_loss = 0.0

target.shape torch.Size([64]) tensor([6, 2, 0, 6, 5, 8, 0, 6, 1, 4, 1, 7, 7, 0, 7, 3, 0, 1, 0, 7, 4, 2, 2, 5,
        7, 2, 6, 3, 2, 9, 7, 2, 3, 4, 1, 3, 0, 8, 3, 7, 6, 7, 5, 1, 8, 2, 3, 8,
        4, 2, 5, 4, 5, 0, 8, 0, 7, 3, 2, 6, 2, 7, 7, 5])
output.shape torch.Size([64, 10]) tensor([ 0.0467,  0.1483,  0.0750, -0.0185, -0.0073,  0.0327,  0.0647,  0.0828,
         0.0023, -0.0072], grad_fn=<SelectBackward>)
